In [121]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import random
import time
import tqdm
from collections import OrderedDict
import cloudscraper
import time

In [142]:
def preprocess_data(soup):
    prices = soup.find_all(attrs={"data-qa" : "POSTING_CARD_PRICE"})
    names = soup.find_all(attrs={"class" : "sc-ge2uzh-0 bzGYzE"})
    urls = soup.find_all(attrs={"data-qa" : "posting PROPERTY"})
    features = soup.find_all(attrs={"data-qa" : "POSTING_CARD_FEATURES"})


    prices = [x.next_element for x in prices]
    names = [x.next_element for x in names]
    urls = [host_url + x['data-to-posting'] for x in urls]

    total_areas = [x.findAll('span')[1].contents[2] for x in features]
    util_areas = [x.findAll('span')[3].contents[2] for x in features]
    rooms = [x.findAll('span')[5].contents[2] for x in features]
    try:
        toilets = [x.findAll('span')[7].contents[2] for x in features]
    except:
        toilets = []
        
    df = pd.DataFrame(list(zip(prices, names, urls, total_areas, util_areas, rooms, toilets)),
              columns=['price','name', 'url',' total_area', "util_area", "rooms", "toilets"])
    
    return df

In [12]:
#headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
headers = OrderedDict({
    'Accept-Encoding': 'gzip, deflate, br',
    'Host': "grimaldis.myguestaccount.com",
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'
})

In [132]:
host_url = "https://www.wimoveis.com.br"
asa_n = "https://www.wimoveis.com.br/apartamentos-venda-asa-norte-brasilia-particular.html"
asa_s = "https://www.wimoveis.com.br/apartamentos-venda-asa-sul-brasilia-particular.html"
noroeste = "https://www.wimoveis.com.br/apartamentos-venda-noroeste-brasilia-particular.html"
sudoeste = "https://www.wimoveis.com.br/apartamentos-venda-sudoeste-brasilia-particular.html"
aguas_c = "https://www.wimoveis.com.br/apartamentos-venda-aguas-claras-df-particular.html"

regions = [asa_n, asa_s, noroeste, sudoeste, aguas_c]

In [133]:
# response = requests.get(asa_n, headers=headers)
# response

In [134]:
scraper = cloudscraper.create_scraper(delay=10,   browser={'custom': 'ScraperBot/1.0',})

# req = scraper.get(asa_n)
# req

In [143]:
for url in regions:
    print("getting this:", url)
    res = scraper.get(url)
    soup = BeautifulSoup(res.content,'html')
    df = preprocess_data(soup)
    df.to_csv(f'../data/{url.replace(host_url+"/","")}.csv', index=False)
    time.sleep(2)

getting this: https://www.wimoveis.com.br/apartamentos-venda-asa-norte-brasilia-particular.html
getting this: https://www.wimoveis.com.br/apartamentos-venda-asa-sul-brasilia-particular.html
getting this: https://www.wimoveis.com.br/apartamentos-venda-noroeste-brasilia-particular.html
getting this: https://www.wimoveis.com.br/apartamentos-venda-sudoeste-brasilia-particular.html
getting this: https://www.wimoveis.com.br/apartamentos-venda-aguas-claras-df-particular.html
